# Hidden Markov Models

# 1.1 Forward algorithm, 1.2 Viterbi algorithm

In [1]:
import numpy as np

states = ["red", "green", "blue"] # given states
initial_prob = [1.0, 0, 0 ] # given probability

# given A matrix
A = [[0.6,0.4,0],
    [0,0.3,0.7],
    [0,0,1]]
# combined b0, b1, b2
B = [[0.8, 0.65, 0.20],
    [0.15, 0.10, 0.30],
    [0.05, 0.25, 0.50]]
observation_seq = [1,1,3,2,1,3] #red red blue green red blue

In [2]:
def emissionProb(state_id, observation):
    result = B[state_id][observation-1]
    return result

In [3]:
def HMM(states, initial_prob, B, observation_seq, A):
    table = np.zeros((len(states),len(observation_seq)))
    back_trace_table = np.zeros((len(states),len(observation_seq)))
    
    #Initialization
    for i in range(len(states)):
        table[i][0] = initial_prob[i]*emissionProb(i,observation_seq[0])
    
    #Forward Algorithm - Recursion
    for observation_id in range(1,len(observation_seq),1):
        for state_id in range(len(states)):
            paths = []
            for path_id in range(len(states)):
                element1 = table[path_id][observation_id-1]
                element2 = A[path_id][state_id]
                element3 = emissionProb(state_id, observation_seq[observation_id])
                path_element = element1*element2*element3
                paths.append(path_element)
            table[state_id][observation_id] = sum(paths)
            back_trace_table[state_id][observation_id] = np.argmax(paths)

    #Viterbi Decoding
    max_state_id = int(np.argmax(table[:,-1]))
    state_pointer = int(back_trace_table[max_state_id][-1])
    inverse_best_seq = states[max_state_id]
    for observation_id in range(len(observation_seq)-2,-1,-1):
        inverse_best_seq += states[state_pointer]
        state_pointer = int(back_trace_table[state_pointer][observation_id])
    best_sequence = inverse_best_seq[::-1]
    #print("inverse best seq", inverse_best_seq)
    prob = [table[i][-1] for i in range(table.shape[0])]
    probability_of_observation_seq = sum(prob)
    
    print("Forward table:\n",table)
    print("")
    print("Backtrace Table:\n",back_trace_table)
    print("")
    print("Best Hidden Sequence",best_sequence)
    print("")
    print("Probability of Observating the sequence “red red blue green red blue”:",probability_of_observation_seq)
    
    return probability_of_observation_seq
probability_of_observation_seq = HMM(states, initial_prob, B, observation_seq, A)

Forward table:
 [[8.00000000e-01 3.84000000e-01 4.60800000e-02 1.79712000e-02
  8.62617600e-03 1.03514112e-03]
 [0.00000000e+00 4.80000000e-02 5.04000000e-02 3.35520000e-03
  1.22925600e-03 1.14577416e-03]
 [0.00000000e+00 0.00000000e+00 1.68000000e-02 1.30200000e-02
  7.68432000e-04 8.14455600e-04]]

Backtrace Table:
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 2. 1.]]

Best Hidden Sequence derderderderderneerg

Probability of Observating the sequence “red red blue green red blue”: 0.00299537088


# 1.3 N-Gram Language Models

# Bigram Language Model to compute the probability of the sencence: FROM DENVER TO SEATTLE

In [4]:
import math
import nltk
from nltk.tokenize import sent_tokenize as st
from nltk.tokenize import word_tokenize as wt

In [5]:
unigram_dict = {}
bigram_dict = {}
trigram_dict = {}
SENTENCE_START = "<s>"
SENTENCE_END = "</s>"
unigram_corpus_length = 0
bigram_corpus_length = 0
trigram_corpus_length = 0
unigram_unique_words = 0
bigram_unique_words = 0
trigram_unique_words = 0
bigram_unique_set = set()
trigram_unique_set = set()

In [6]:
def calcgram(s):
    return len(s) - 2

def calculate_unigram(s):
    res = 0
    for sentence in s:
        res += len(sentence) - 2
    return res

def calculate_bigram(s):
    res = 0
    for sentence in s:
        res += len(sentence) - 1
    return res

def calculate_trigram(s):
    res = 0
    for sentence in s:
        res += len(sentence) - 2
    return res

def preprocess_sentence(s):
    new_words = [SENTENCE_START]
    for word in wt(s):
        new_words.append(word)
    new_words.append(SENTENCE_END)
    return new_words

def preprocess_sentences(s):
    sentences = st(s)
    new_sentences = []
    for sentence in sentences:
        new_sentences.append(preprocess_sentence(sentence[:-1])[:])
    return new_sentences

def preprocess(s):
    unigram(s)
    bigram(s)
    trigram(s)
    
def unigram(s):
    '''Unigram model'''
    global unigram_corpus_length
    for sentence in s:
        for word in sentence:
            try:
                unigram_dict[word] += 1
            except:
                unigram_dict[word] = 1
            if(word != SENTENCE_START and word != SENTENCE_END):
                unigram_corpus_length += 1
    unigram_unique_words = len(unigram_dict) - 2  

def bigram(s):
    '''Bigrm model'''
    global bigram_corpus_length
    for sentence in s:
        previous_word = None
        for word in sentence:
            if(previous_word != None):
                try:
                    bigram_dict[(previous_word,word)] += 1
                except:
                    bigram_dict[(previous_word,word)] = 1
                if previous_word != SENTENCE_START and word != SENTENCE_END:
                    bigram_unique_set.add((previous_word, word))
            previous_word = word
    bigram_unique_words = len(bigram_dict)
                    
def trigram(s):
    '''Trigram model'''
    global trigram_corpus_length
    for sentence in s:
        first_word = None
        second_word = None
        for word in sentence:
            if(first_word != None and second_word != None):
                try:
                    trigram_dict[(first_word,second_word,word)] += 1
                except:
                    trigram_dict[(first_word,second_word,word)] = 1
                if first_word != SENTENCE_START and word != SENTENCE_END:
                    trigram_unique_set.add((first_word,second_word,word))
            first_word = second_word
            second_word = word
    trigram_unique_words = len(trigram_dict)
                    

def probability_unigram(word,smooth):
    try:
        result_numerator = unigram_dict[word]
    except:
        result_numerator = 0
    try:
        result_denumerator = unigram_unique_words
    except:
        result_denumerator = 0
    if(smooth):
        result_numerator += 1
        result_denumerator += 1
    return float(result_numerator) / float(result_denumerator)

def probability_bigram(previous_word,word,smooth):
    try:
        result_numerator = bigram_dict[(previous_word,word)]
    except:
        result_numerator = 0
    try:
        result_denumerator = unigram_dict[previous_word]
    except:
        result_denumerator = 0
    if(smooth):
        result_numerator += 1
        result_denumerator += 1
    if(result_numerator == 0 or result_denumerator ==0):
        return 0.0
    return float(result_numerator) / float(result_denumerator)

def probability_trigram(first_word,second_word,word,smooth):
    try:
        result_numerator = trigram_dict[(first_word,second_word,word)]
    except:
        result_numerator = 0
    try:
        result_denumerator = bigram_dict[(first_word,second_word)]
    except:
        result_denumerator = 0
    if(smooth):
        result_numerator += 1
        result_denumerator += 1
    if(result_numerator == 0 or result_denumerator ==0):
        return 0.0
    return float(result_numerator) / float(result_denumerator)

def sentence_probability(gram_type,sentence,smooth):
    '''Function to calculate the sentence probability'''
    result = 0.0
    if(gram_type == "Unigram"):
        for word in sentence :
            if(word != SENTENCE_START and word != SENTENCE_END):
                if(result == 0.0):
                    result = probability_unigram(word,smooth)
                else:
                    result *= probability_unigram(word,smooth)

    elif(gram_type == "Bigram"):
        previous_word = None
        for word in sentence:
            if(previous_word != None  and word != SENTENCE_END):
                if(result == 0.0):
                    result = probability_bigram(previous_word,word,smooth)
                else:
                    result *= probability_bigram(previous_word,word,smooth)
            previous_word = word

    elif(gram_type =="Trigram"):
        first_word = None
        second_word = None
        for word in sentence:
            if(first_word != None and word != SENTENCE_END):
                if(result == 0.0):
                    result = probability_trigram(first_word,second_word,word,smooth)
                else:
                    result *= probability_trigram(first_word,second_word,word,smooth)
            previous_word = word
    return result

In [7]:
train_sentences = "I WANT TO GO FROM DENVER TO BOSTON TOMORROW MORNING. I WOULD LIKE TO GO TO SEATTLE THIS AFTERNOON. DEPARTING SEATTLE. IN THE LATE MORNING. LATE MORNING GOING TO DENVER FROM BOSTON. FROM DENVER LEAVING IN THE MORNING ARRING IN BOSTON."
sentences = preprocess_sentences(train_sentences)
preprocess(sentences)
input_sentences = preprocess_sentence("FROM DENVER TO SEATTLE ")
print('The sentence probability is:',sentence_probability("Bigram",input_sentences,False))

The sentence probability is: 0.007407407407407408
